In [4]:
import sqlite3
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# Define number of patients and appointments
n_patients = 1000
n_appointments = 2500

# Generate admission and discharge dates with time stamps for patients
admission_dates = pd.date_range('2020-01-01', '2023-12-31', periods=n_patients).strftime('%Y-%m-%d %H:%M:%S')
discharge_dates = pd.date_range('2020-01-05', '2023-12-31', periods=n_patients).strftime('%Y-%m-%d %H:%M:%S')

# Create Patients DataFrame
patients = pd.DataFrame({
    'patient_id': np.arange(1, n_patients + 1),
    'first_name': np.random.choice(
        ['Lesley', 'Mary', 'Roseline', 'Edwine', 'Brandon', 'Franklin', 'Synnickc', 'Peter'], 
        n_patients
    ),
    'last_name': np.random.choice(
        ['Fokou', 'Nima', 'Tangmo', 'Malla', 'Nfor', 'Samma', 'Nken', 'Tonka'], 
        n_patients
    ),
    'gender': np.random.choice(['Male', 'Female'], n_patients, p=[0.49, 0.51]),
    'age': np.clip(np.random.normal(45, 20, n_patients), 0, 80).astype(int),
    'admission_date': admission_dates,
    'discharge_date': discharge_dates
})

# Introduce 15% missing discharge dates to simulate incomplete records
missing_indices = np.random.choice(patients.index, int(0.15 * n_patients), replace=False)
patients.loc[missing_indices, 'discharge_date'] = None

# Create Medical_Staff DataFrame (50 rows)
medical_staff = pd.DataFrame({
    'staff_id': np.arange(1, 51),
    'full_name': [f'Dr. {name}' for name in np.random.choice(['Rosian', 'Lee', 'Sandrine'], 50)],
    'department': np.random.choice(['ICU', 'Surgery', 'Pediatrics', 'Cardiology'], 50),
    'salary': np.round(np.random.lognormal(10.5, 0.3, 50), 2)
})

# Generate Appointments DataFrame (2500 rows)
appointments_list = []
for _ in range(n_appointments):
    appointment_date = pd.to_datetime('2023-01-01') + pd.Timedelta(days=np.random.randint(0, 365))
    appointments_list.append({
        'patient_id': np.random.choice(patients['patient_id']),
        'staff_id': np.random.choice(medical_staff['staff_id']),
        'appointment_date': appointment_date.strftime('%Y-%m-%d %H:%M:%S'),
        'priority': np.random.choice(['Low', 'Medium', 'High'], p=[0.6, 0.3, 0.1])
    })
appointments = pd.DataFrame(appointments_list)

# Generate unique appointment_id for each appointment
appointments['appointment_id'] = np.arange(1, len(appointments) + 1)
# Reorder columns so that appointment_id is first
appointments = appointments[['appointment_id', 'patient_id', 'staff_id', 'appointment_date', 'priority']]

# Create a new SQLite database named 'healthcare.db'
conn = sqlite3.connect('healthcare.db')
cursor = conn.cursor()

# Enable foreign key constraint enforcement
cursor.execute("PRAGMA foreign_keys = ON")

# Drop tables if they already exist
cursor.execute("DROP TABLE IF EXISTS Appointments")
cursor.execute("DROP TABLE IF EXISTS Medical_Staff")
cursor.execute("DROP TABLE IF EXISTS Patients")


# Create Patients Table
cursor.execute('''
CREATE TABLE Patients (
    patient_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    gender TEXT CHECK(gender IN ('Male', 'Female')),
    age INTEGER,
    admission_date TEXT,
    discharge_date TEXT
)
''')

# Create Medical_Staff Table

cursor.execute('''
CREATE TABLE Medical_Staff (
    staff_id INTEGER PRIMARY KEY,
    full_name TEXT NOT NULL,
    department TEXT,
    salary REAL CHECK(salary >= 0)
)
''')

# Create Appointments Table

cursor.execute('''
CREATE TABLE Appointments (
    appointment_id INTEGER PRIMARY KEY,
    patient_id INTEGER,
    staff_id INTEGER,
    appointment_date TEXT,
    priority TEXT CHECK(priority IN ('Low', 'Medium', 'High')),
    FOREIGN KEY(patient_id) REFERENCES Patients(patient_id),
    FOREIGN KEY(staff_id) REFERENCES Medical_Staff(staff_id)
)
''')

# Insert data into the SQLite tables using pandas to_sql
patients.to_sql('Patients', conn, if_exists='append', index=False)
medical_staff.to_sql('Medical_Staff', conn, if_exists='append', index=False)
appointments.to_sql('Appointments', conn, if_exists='append', index=False)

conn.commit()

# Verify by reading back a few rows from each table
patients_check = pd.read_sql_query("SELECT * FROM Patients LIMIT 5", conn)
staff_check = pd.read_sql_query("SELECT * FROM Medical_Staff LIMIT 5", conn)
appointments_check = pd.read_sql_query("SELECT * FROM Appointments LIMIT 5", conn)
conn.close()

print("Patients Sample:")
print(patients_check)
print("\nMedical Staff Sample:")
print(staff_check)
print("\nAppointments Sample:")
print(appointments_check)


Patients Sample:
   patient_id first_name last_name  gender  age       admission_date  \
0           1   Synnickc     Samma    Male   27  2020-01-01 00:00:00   
1           2     Edwine      Nken  Female   28  2020-01-02 11:04:30   
2           3    Brandon     Fokou  Female   40  2020-01-03 22:09:00   
3           4   Synnickc     Fokou  Female   52  2020-01-05 09:13:30   
4           5   Roseline     Fokou  Female   63  2020-01-06 20:18:01   

        discharge_date  
0  2020-01-05 00:00:00  
1  2020-01-06 10:58:44  
2  2020-01-07 21:57:28  
3                 None  
4  2020-01-10 19:54:57  

Medical Staff Sample:
   staff_id     full_name  department    salary
0         1    Dr. Rosian  Pediatrics  25053.58
1         2       Dr. Lee  Pediatrics  37527.25
2         3       Dr. Lee  Pediatrics  54100.15
3         4       Dr. Lee  Cardiology  39892.97
4         5  Dr. Sandrine  Pediatrics  30274.13

Appointments Sample:
   appointment_id  patient_id  staff_id     appointment_date priori